In [1]:
from model import SpiceMixPlus
from load_data import load_anndata, save_anndata
from pathlib import Path
import torch
import numpy as np
import itertools
from torch import nn
import importlib
from tqdm import trange

In [2]:
def reload_all():
    import model, load_data, estimate_parameters, estimate_weights, util, initialization, sample_for_integral
    importlib.reload(util)
    importlib.reload(load_data)
    importlib.reload(sample_for_integral)
    importlib.reload(estimate_parameters)
    importlib.reload(estimate_weights)
    importlib.reload(initialization)
    importlib.reload(model)

reload_all()

In [14]:
repli_list = ["151673", "151674"]
context = dict(device='cuda:0', dtype=torch.float32)

K = 20

model = SpiceMixPlus(
    K=20, lambda_Sigma_x_inv=1e-12,
    repli_list=[0, 1],
    context=context,
    #path2result="../../data/spatialLIBD/results/phenotype_prediction_K20_lambda_1e-12.hdf5",
#     context_Y=dict(dtype=torch.float32, device='cpu'),
    context_Y=context,
)   
model.load_dataset(path2dataset, data_format="raw")

NameError: name 'context' is not defined

0      1
1      1
2      1
3      1
4      1
      ..
495    1
496    1
497    1
498    1
499    1
Name: batch, Length: 500, dtype: category
Categories (1, object): ['1']

In [3]:
path2dataset = Path('../../data/spatialLIBD')

In [4]:
!head ../../data/spatialLIBD/files/meta_151675.csv

coor X,coor Y,real X,real Y,cell type
0.0,16.0,115.481552132,172.547260668,L1
86.60254,102.0,386.993712698,437.038718922,L3
102.190994,19.0,433.61837677,179.522958478,WM
24.24871,94.0,192.709277564,413.681382384,L3
81.40639,13.0,368.81189389,161.386144172,L5
126.439705,43.0,509.67598515,253.330341758,WM
105.6551,97.0,446.174632828,421.107125214,L3
77.94228,115.0,360.351047514,477.542770722,L2
72.74613,28.0,342.169228706,208.055812746,L5


In [ ]:
repli_list = ["151673", "151674"]
context = dict(device='cuda:0', dtype=torch.float32)

K = 20

model = SpiceMixPlus(
    K=K, lambda_Sigma_x_inv=1e-12,
    repli_list=repli_list,
    context=context,
    #path2result="../../data/spatialLIBD/results/phenotype_prediction_K20_lambda_1e-12.hdf5",
#     context_Y=dict(dtype=torch.float32, device='cpu'),
    context_Y=context,
)   
model.load_dataset(path2dataset)
model.meta['cell type'] = model.meta['cell type'].astype('category')

phenotype2predictor = {}
df = model.meta
# cell type
label_raw = df['cell type'].astype(str)
label_raw[label_raw.isna()] = "NA"
label_a2i = dict(zip(np.unique(np.nan_to_num(label_raw.values).astype(str)), itertools.count()))
label = list(map(label_a2i.get, label_raw))
df['cell type encoded'] = np.array(label, dtype=int)

predictor = nn.Linear(K, len(set(label)))
predictor = predictor.to(context['device'])
predictor.weight.data.normal_()
predictor.bias.data.normal_()
optimizer = torch.optim.Adam(predictor.parameters(), lr=1e-2, weight_decay=1e-2)

# compute_loss = nn.CrossEntropyLoss()
def compute_loss(yhat, y, mode="train"):
    if mode == 'train':
         return torch.nn.functional.cross_entropy(yhat, y, reduction='mean')
    elif mode == 'eval':
         return torch.nn.functional.cross_entropy(yhat, y, reduction='sum') * 10
    else:
        raise NotImplementError

phenotype2predictor['cell type encoded'] = (predictor, optimizer, compute_loss)
# del label_raw, label, predictor, optimizer, compute_loss
model.register_phenotype(phenotype2predictor)
print(model.Ys)
model.phenotypes[1]['cell type encoded'] = None

model.initialize(
    method='svd',
)   
# evaluate(model)
for iiter in trange(20):
    model.estimate_parameters(iiter=iiter, use_spatial=[False]*model.num_repli)
    model.estimate_weights(iiter=iiter, use_spatial=[False]*model.num_repli)
# evaluate(model)
model.initialize_Sigma_x_inv()

for iiter in range(1, 500):
    model.estimate_parameters(iiter=iiter, use_spatial=[True]*model.num_repli)
    model.estimate_weights(iiter=iiter, use_spatial=[True]*model.num_repli)

[tensor([[0.0078, 0.0000, 0.0078,  ..., 0.0118, 0.0422, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0242, 0.0403, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0252, 0.0396, 0.0000],
        ...,
        [0.0000, 0.0159, 0.0000,  ..., 0.0159, 0.0443, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0414, 0.0000],
        [0.0000, 0.0000, 0.0095,  ..., 0.0095, 0.0417, 0.0000]],
       device='cuda:0'), tensor([[0.0000, 0.0000, 0.0062,  ..., 0.0097, 0.0399, 0.0000],
        [0.0000, 0.0000, 0.0115,  ..., 0.0188, 0.0369, 0.0000],
        [0.0000, 0.0104, 0.0000,  ..., 0.0104, 0.0370, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0394, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0338, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0182, 0.0394, 0.0000]],
       device='cuda:0')]


Updating `cell type encoded` predictor:   0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

Updating M:   0%|          | 0/10000 [00:00<?, ?it/s]

Updating `cell type encoded` predictor:   0%|          | 0/10000 [00:00<?, ?it/s]

...

Updating Z w/ nbrs w/ ph.:   0%|          | 0/3639 [00:00<?, ?it/s]

Updating Z w/ nbrs w/ ph.:   0%|          | 0/3639 [00:00<?, ?it/s]

Updating weight w/ neighbors:   0%|          | 0/1000 [00:00<?, ?it/s]

Updating Σx-1:   0%|          | 0/1000 [00:00<?, ?it/s]

Updating M:   0%|          | 0/10000 [00:00<?, ?it/s]

In [105]:
print("Done")

Done


In [109]:
model.meta

,coor X,coor Y,real X,real Y,cell type,repli,cell type encoded,label SpiceMixPlus predictor
0,86.602540,102.0,381.098123,440.639079,L3,151673,2,2
1,5.196152,43.0,126.327637,259.630972,L1,151673,0,1
2,102.190994,19.0,427.767792,183.078314,WM,151673,7,7
3,24.248710,94.0,186.813688,417.236738,L3,151673,2,2
4,74.478180,9.0,341.269139,152.700275,L5,151673,4,5
...,...,...,...,...,...,...,...,...
3668,100.458950,42.0,425.787593,251.800189,WM,151674,7,7
3669,103.923050,30.0,436.273642,214.536461,WM,151674,7,7
3670,77.942280,27.0,355.355548,205.850592,L6,151674,5,5
3671,126.439705,41.0,506.615679,248.109820,WM,151674,7,7


In [111]:
from util import evaluate_prediction
display(evaluate_prediction(model.meta, key_pred='label SpiceMixPlus predictor'))

,acc,f1 micro,f1 macro,ari
151673,0.0,0.0,0.0,0.480078
151674,0.0,0.0,0.0,0.131621


In [112]:
dir(model)

['Es',
 'Es_isempty',
 'GG',
 'Gs',
 'K',
 'M',
 'M_bar',
 'M_constraint',
 'Ms',
 'Ns',
 'Sigma_x_inv',
 'Ss',
 'X_constraint',
 'Xs',
 'Ys',
 'Z_optimizers',
 'Zs',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'betas',
 'context',
 'context_Y',
 'dropout_mode',
 'estimate_M',
 'estimate_parameters',
 'estimate_phenotype_predictor',
 'estimate_sigma_yx',
 'estimate_weights',
 'genes',
 'get_M',
 'initialize',
 'initialize_Sigma_x_inv',
 'lambda_M',
 'lambda_Sigma_x_inv',
 'load_dataset',
 'meta',
 'meta_repli',
 'num_repli',
 'optimizer_Sigma_x_inv',
 'pairwise_potential_mode',
 'phenotype_predictors',
 'phenotypes',
 'prior_x_modes',
 'prior_xs',
 'random_state',


In [125]:
model.Xs

[tensor([[2.3703e-09, 2.3703e-09, 2.3703e+01,  ..., 2.3703e-09, 2.3703e-09,
          2.3703e-09],
         [1.5231e-09, 1.0158e+01, 1.5231e-09,  ..., 1.5231e-09, 1.5079e+00,
          1.5231e-09],
         [1.8996e-09, 1.8996e-09, 1.8996e-09,  ..., 1.8996e-09, 1.8996e-09,
          1.8996e-09],
         ...,
         [1.7467e-09, 1.7467e-09, 1.7467e-09,  ..., 1.7467e-09, 1.7467e-09,
          1.7467e-09],
         [1.7548e-09, 1.7548e-09, 1.7548e-09,  ..., 1.7548e-09, 1.7548e-09,
          1.7548e-09],
         [2.2305e-09, 1.3652e+01, 2.2305e-09,  ..., 2.2305e-09, 2.2305e-09,
          2.2305e-09]], device='cuda:0'),
 tensor([[2.1522e-09, 2.1522e-09, 2.1522e-09,  ..., 2.1522e-09, 2.1522e-09,
          2.1522e-09],
         [1.9892e-09, 1.9892e-09, 1.9892e-09,  ..., 1.9892e-09, 1.9892e-09,
          1.9892e-09],
         [1.8481e-09, 1.8481e-09, 1.8481e-09,  ..., 1.8481e-09, 1.8481e-09,
          1.8481e-09],
         ...,
         [1.3991e-09, 1.3991e-09, 1.3991e-09,  ..., 1.3991e-09

In [133]:
import pickle

with open("../../data/spatialLIBD/results/phenotype_prediction_K20_Sigma_x_inv.pkl", "wb") as f:
    pickle.dump(model.Sigma_x_inv.cpu().detach().numpy(), f)
    
with open("../../data/spatialLIBD/results/phenotype_prediction_K20_M.pkl", "wb") as f:
    pickle.dump(model.M.cpu().detach().numpy(), f)
    
with open("../../data/spatialLIBD/results/phenotype_prediction_K20_Xs.pkl", "wb") as f:
    pickle.dump([X.cpu().detach().numpy() for X in model.Xs], f)

In [118]:
!ls ../../data/spatialLIBD/results/phenotype_prediction_K20_Sigma_x_inv.npy

2_SpiceMix_K20_1e-12_rs1.h5
phenotype_prediction_K20_lambda_1e-12.hdf5
results_with_spatial_K_15_lambda_1e-12_single_fov_random_seed_11.hdf5
results_with_spatial_K_15_lambda_1e-12_single_fov_random_seed_12.hdf5
results_with_spatial_K_15_lambda_1e-12_single_fov_random_seed_13.hdf5
results_with_spatial_K_15_lambda_1e-12_single_fov_random_seed_14.hdf5
results_with_spatial_K_15_lambda_1e-12_single_fov_random_seed_15.hdf5
results_with_spatial_K_15_lambda_1e-12_single_fov_random_seed_16.hdf5
results_with_spatial_K_15_lambda_1e-8_single_fov_random_seed_11.hdf5
results_with_spatial_K_15_lambda_1e-8_single_fov_random_seed_12.hdf5
results_with_spatial_K_20_lambda_1e-12_four_fovs_random_seed_11.hdf5
results_with_spatial_K_20_lambda_1e-12_four_fovs_random_seed_12.hdf5
results_with_spatial_K_20_lambda_1e-12_four_fovs_random_seed_13.hdf5
results_with_spatial_K_20_lambda_1e-12_four_fovs_random_seed_14.hdf5
results_with_spatial_K_20_lambda_1e-12_four_fovs_random_seed_15.hdf5
results_with_spatial_K_20_